In [26]:
import pandas as pd
import numpy as np
df_faces = pd.read_csv('UIdata/faces.csv')
df_attr = pd.read_excel('UIdata/alphapolitica_faces_attributes_coords.xlsx')

In [115]:
'''Generate folium plot'''

loc_dict = dict(zip(df_attr.image_id, df_attr.coords))
per_dict = dict(zip(df_attr.image_id, df_attr.Name))
total_faces = df_faces.columns[6:]

df_fid_list =[]
for f_id in total_faces:
    df_face = df_faces[df_faces[f_id]> 0][['upload_date','sentiment','view_count']]
    df_face['lat'] = float(loc_dict[f_id].split(',')[0])
    df_face['long'] = float(loc_dict[f_id].split(',')[1])
    df_face['name'] = per_dict[f_id]
    df_face['constituency'] = per_dict[f_id]
    df_fid_list.append(df_face[['lat','long','upload_date','name','sentiment','view_count']])
df_folium_plot = pd.concat(df_fid_list)
df_folium_plot = df_folium_plot.reset_index()
del df_folium_plot['index']
df_folium_plot.to_csv('plots_data/dashboard_folium.csv',index= False)

## Folium Map

In [121]:
import folium
from folium.plugins import HeatMap
df_folium_plot = pd.read_csv('plots_data/dashboard_folium.csv')

date_start = '2023-01-01'
date_end = '2024-01-01'

df_folium_sorted = df_folium_plot[(df_folium_plot.upload_date >= date_start) & (df_folium_plot.upload_date <= date_end)]

df_folium_sorted['heat'] = df_folium_sorted['view_count']
df_folium_sorted['heat'] = (df_folium_sorted['heat'] - df_folium_sorted['heat'].min())/(df_folium_sorted['heat'].max() - df_folium_sorted['heat'].min())
df_folium_sorted = df_folium_sorted[['lat','long','heat']]
df_folium_sorted = df_folium_sorted.groupby(['lat','long']).mean().reset_index()
lats_longs_weight = df_folium_sorted.values
map_obj = folium.Map(location = [16.315487, 80.232213], zoom_start = 8,)
HeatMap(lats_longs_weight,min_opacit=1,radius=10).add_to(map_obj)

df_markers = df_folium_plot[['lat','long','name']].drop_duplicates()
for i in range(len(df_markers)):
    folium.CircleMarker([df_markers['lat'].values[i], df_markers['long'].values[i]], 
                popup=df_markers['name'].values[i],radius=3,color ='crimson'
                ).add_to(map_obj)
map_obj

